In [1]:
using Plots, Interact
gr()

Plots.GRBackend()

In [2]:
include("discs_in_box.jl")

dynamics

# Important Notice.
In the paper the table is wider than higher. Here is the other way around. Simply rotate your monitor, change w for h, a for b, and everything is fine

In [14]:
w, h = 1, 1.5
rvmax= w/4
rmax=(h+w-√(2*h*w))/2
r = 0.23

@time times, positions, velocities, collision_types = dynamics(w, h, r, 100000);

  0.141599 seconds (1.38 M allocations: 106.418 MiB, 50.27% gc time)


In [15]:
rmax

0.3839745962155614

In [4]:
showall(norm.(velocities)[1:100])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

The above shows that we are using $v^2=1$ or $E=1/2$

In [5]:
showall(collision_types[1:100])

[3, 1, -2, -3, 4, 3, -1, -3, 5, 4, 5, 3, 1, -3, -2, 3, -1, 5, -1, -2, 4, 5, -2, 3, 4, -1, 5, -1, 4, 3, 1, -2, 5, -2, -1, -3, 1, 5, 4, 5, -2, 5, 4, -1, 5, 3, -2, -3, 3, 5, -1, 4, 5, 1, -2, 3, 4, -1, 5, 4, -1, 3, 5, -1, 3, 5, 3, -1, 5, -1, 4, 3, -2, 1, 5, 4, -1, -2, 5, -2, 5, 3, 4, 5, 1, 4, -3, -2, 3, -1, 5, -1, 4, -2, 5, 3, -2, 5, 4, -3]

The function `dynamics` returns data on the collisions that have occurred.

In [6]:
function draw_disc!(x, y, r)
    θs = [0:0.1:2π; 0]
    xs = [x + r*cos(θ) for θ in θs]
    ys = [y + r*sin(θ) for θ in θs]
    
    plot!(xs, ys)
end

draw_disc! (generic function with 1 method)

In [7]:
@manipulate for i in slider(1:length(positions), value=1)
    x1, y1, x2, y2 = positions[i]
    
    a = w/2
    b = h/2
    
    plot([-a, a, a, -a, -a], [-b, -b, b, b, -b], aspectratio=1, leg=false, xlim=(-0.5,0.5), ylim=(-0.75,0.75))
    draw_disc!(x1, y1, r) 
    draw_disc!(x2, y2, r) 
end

Interact.Slider{Int64}(1: "input" = 1 Int64 , "", 1, 1:100000, "horizontal", true, "d", true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75

# Hopping

The data from the simulation directly gives disc and wall collision times.
The other times that we require are hopping times, both horizontal, when $x_1 = x_2$, and vertical, when $y_1 = y_2$. 

These can be detected from the data when $x_2 - x_1$ changes sign between two consecutive collisions.
Using the relative velocity $u_2 - u_1$, the hopping time when $x_2 - x_1 = 0$ can be recovered.

In [10]:
"""
Calculate the times at which horizontal hops occur
"""
function horizontal_hopping_times(times, positions, velocities)
    Δxs = [x[3] - x[1] for x in positions]  # x_2 - x_1
    Δus = [v[3] - v[1] for v in velocities]  # u_2 - u_1;
    
    # indices where there is a hop between collisions i and i+1: 
    horiz_hop_indices = find( sign(Δxs[i]) != sign(Δxs[i+1]) for i in 1:length(positions)-1 );  
    
    # x + t*u = 0   so   t = -x/u
    horiz_hopping_times = times[horiz_hop_indices] - (Δxs[horiz_hop_indices] ./ Δus[horiz_hop_indices])
    
    return horiz_hopping_times
end

horizontal_hopping_times

Analytical expression for horizontal hopping:

In [12]:
cte=3π / 4
#In this simulation E=1/2 and abs(v)=1, check your normalization
areahop(a,b,r)=8*√(2)*a*(b-r)^2
areacolfree(a,b,r)=16*√(2)*r*(π/2*a*b-(a+b)*r+r^2/2)


volbox(a,b)=16*a^2*b^2

negvolhopfree(a,b,r)=16*π*a*b*r^2-64/3*(a+b)*r^3+8*r^4

volneguglypart(a,b,r,θ)=8*r^2*(4*a*b*θ+r^2*(sin(θ)^2)+8/3*r*(a*cos(θ)-b*sin(θ)))
volneguppertriangle(a,b,r)=8/3*b^2*(4*a*√(r^2-b^2)+b^2-r^2)
volneglowertriangle(a,b,r)=volneguppertriangle(b,a,r)

volhopfree(a,b,r)=volbox(a,b)-negvolhopfree(a,b,r)    
horiz_hop_analytical(a, b, r) =cte*volhopfree(a,b,r)/areahop(a,b,r)   
    
#horiz_hop_analytical(a, b, r) = (3π / (2*√(2))) * (2*a^2*b^2 - 2π*a*b*r^2 + ((a+b)/3)*(2*r)^3 - r^4) / (a*√(2)*(b-r)^2)

#disc_collision_analytical(a, b, r) = cte* (2*a^2*b^2 - 2π*a*b*r^2 + ((a+b)/3)*(2*r)^3 - r^4) /(2π*a*b*r - 4*(a+b)*r^2 + 2r^3 )
disc_collision_analytical(a, b, r) = cte* volhopfree(a,b,r) / areacolfree(a,b,r)


disc_collision_analytical (generic function with 1 method)

In [36]:
function simulation(w, h, num_collisions=10^6)
    
    num_horiz_hop_data = Float64[]
    exact_horiz_hop_data = Float64[]
    num_disc_collision_data = Float64[]
    exact_disc_collision_data = Float64[]
    
    rs = 0.005:0.005:0.245
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        
        horiz_hopping_times = horizontal_hopping_times(times, positions, velocities)

        push!(num_horiz_hop_data, mean(diff(horiz_hopping_times)))   # diff gives inter-hop times
        push!(exact_horiz_hop_data, horiz_hop_analytical(a, b, r))
        
        
        disc_collision_times = times[collision_types .== 5]
        push!(num_disc_collision_data, mean(diff(disc_collision_times)))   # diff gives inter-hop times
        push!(exact_disc_collision_data, disc_collision_analytical(a, b, r))

    end
    
    return rs, num_horiz_hop_data, exact_horiz_hop_data, num_disc_collision_data, exact_disc_collision_data
end

simulation (generic function with 2 methods)

In [37]:
w, h = 1.0, 1.5
rs, num_horiz_hop_data, exact_horiz_hop_data, num_disc_collision_data, exact_disc_collision_data = simulation(w, h);

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 0.235 0.24 0.245 

In [38]:
Plots.scatter(rs, num_disc_collision_data, m=:square, label="numerical")
plot!(rs, exact_disc_collision_data, label="exact")
# Now this is wrong

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 exact

In [39]:
plot(rs, num_horiz_hop_data, m=:square, label="numerical")
plot!(rs, exact_horiz_hop_data, m=:circle, label="exact")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
 
 1.1 
 
 
 1.2 
 
 
 1.3 
 
 
 1.4 
 
 
 1.5 
 
 
 1.6 
 
 
 1.7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 
 
 exact

In [35]:
plot(rs, exact_horiz_hop_data./num_horiz_hop_data, m=:square)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
 
 0.90 
 
 
 0.95 
 
 
 1.00 
 
 
 1.05 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [48]:
num_horiz_hop_data[1:46]

46-element Array{Float64,1}:
 1.73343
 1.66552
 1.70359
 1.68172
 1.71346
 1.70053
 1.68459
 1.68757
 1.70676
 1.70026
 1.69151
 1.66803
 1.68916
 ⋮      
 1.47159
 1.43849
 1.40015
 1.38659
 1.34835
 1.34429
 1.27584
 1.28632
 1.25595
 1.21913
 1.19083
 1.16243

In [38]:
histogram(diff(horiz_hopping_times))

LoadError: [91mUndefVarError: horiz_hopping_times not defined[39m

In [39]:
histogram(diff(horiz_hopping_times), normed=true)

LoadError: [91mUndefVarError: horiz_hopping_times not defined[39m